<a href="https://colab.research.google.com/github/artms-18/ML-Projects/blob/main/Pets_Object_Localization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Using a Resnet50 feature extractor to classify and locate objects within the Oxford-IIIT Pet Dataset

In this notebook, we will achieve this by implementing the following steps:
1. Importing Modules
2. Preprocessing data
3. Defining helper functions to later view the data
4. Creating the model using a pretrained feature extractor, but not including the top so that we can add our own Dense layers
5. Compiling and fitting the model
6. Visualizing the results and looking at accuracy as well as loss on Tensorflow Hub

## Importing Modules

In [14]:
import glob
import os
import csv
import cv2
from zipfile import ZipFile

import xml.etree.ElementTree as ET
import numpy as np
import tensorflow as tf

## Getting the Data

In [15]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [16]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"mikashaw","key":"5ea1167847d0f4828837c42bb91725e5"}'}

In [17]:
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d devdgohil/the-oxfordiiit-pet-dataset

the-oxfordiiit-pet-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
with zipfile.ZipFile('/content/the-oxfordiiit-pet-dataset.zip', 'r') as zip_file:
  zip_file.extractall('/content/the-oxfordiiit-pet-dataset')

## Preprocessing the Data

Right now, the annotations are in an xml file. In order to train the model, we will need to convert the data contained into a numpy array

In [20]:
XML_PATH = '/content/the-oxfordiiit-pet-dataset/annotations/annotations/xmls'
SPLIT_RATIO = 0.8 #while splitting into training and validation

In [24]:
class_names = {} #storing the name pertaining to each class of images
k = 0 #classes will be label encoded
output = [] #this will soon be a list containing the labels for each image (paht, height, width, xmin, xmas, ymax, class_name, class_names[class_name])

xml_files = glob.glob("{}/*xml".format(XML_PATH))
for i, xml_file in enumerate(xml_files):
  tree = ET.parse(xml_file)
  path = os.path.join(XML_PATH, tree.findtext("./filename"))

  height = int(tree.findtext("./size/height"))
  width = int(tree.findtext("./size/width"))
  xmin = int(tree.findtext("./object/bndbox/xmin"))
  ymin = int(tree.findtext("./object/bndbox/ymin"))
  xmax = int(tree.findtext("./object/bndbox/xmax"))
  ymax = int(tree.findtext("./object/bndbox/ymax"))

  basename = os.path.basename(path)
  basename = os.path.splitext(basename)[0]
  class_name = basename[:basename.rfind('_')].lower() #gets the lowercased name of pet (getting rid of jpg number)
  if class_name not in class_names:
    class_names[class_name] = k
    k+=1

  output.append((path, height, width, xmin, ymin, xmax, ymax, class_name, class_names[class_name]))

output.sort(key = lambda tup: tup[-1]) #sorting by class


In [26]:
#taking a look at the amounts of images in each image class

lengths = []
i = 0
last = 0
for j, row in enumerate(output):
  if last == row[-1]: #since earlier, we sorted 'output' from class 0 increasing, we can traverse the list much easier
    i += 1 #this will continue happening until last IS NOT equal to the class label
  else: 
    print(f"class {output[j-1][-2]}: {i} images") #we are doing j-1 because this happens ONLY when we move on to the next class
    lengths.append(i)
    i = 1
    last += 1

print(f"class {output[j-1][-2]}: {i} images") #since the last one still needs to be expressed
lengths.append(i)


class yorkshire_terrier: 100 images
class sphynx: 100 images
class saint_bernard: 99 images
class great_pyrenees: 100 images
class japanese_chin: 100 images
class english_setter: 100 images
class chihuahua: 100 images
class staffordshire_bull_terrier: 100 images
class german_shorthaired: 100 images
class persian: 100 images
class abyssinian: 99 images
class boxer: 100 images
class newfoundland: 100 images
class american_pit_bull_terrier: 100 images
class basset_hound: 100 images
class keeshond: 100 images
class havanese: 100 images
class ragdoll: 99 images
class bengal: 98 images
class birman: 100 images
class english_cocker_spaniel: 100 images
class leonberger: 100 images
class shiba_inu: 100 images
class miniature_pinscher: 100 images
class wheaten_terrier: 100 images
class egyptian_mau: 92 images
class beagle: 100 images
class british_shorthair: 100 images
class bombay: 100 images
class american_bulldog: 100 images
class pomeranian: 100 images
class maine_coon: 100 images
class samo